In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score
import os
import scipy.io
import math

from sklearn.utils import shuffle

from PIL import Image
import requests
from io import BytesIO

from tensorflow.keras.applications import resnet50
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import tensorflow as tf
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from scipy import spatial
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data_raw = pd.read_csv('/kaggle/input/titanic/train.csv')
data_test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = data_raw.copy(deep=True)
data_all = [train, data_test]

In [ ]:
for dataset in data_all:
    dataset.drop(['Ticket', 'Cabin'], axis=1, inplace=True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace=True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
    for c in set(dataset['Pclass']):
        for s in set(dataset['Sex']):
            age_median = dataset[(dataset['Pclass'] == c) & (dataset['Sex'] == s)]['Age'].median()
            dataset.loc[(dataset['Age'].isnull()) & (dataset['Pclass'] == c) & (dataset['Sex'] == s), 'Age'] = age_median
train.drop(['PassengerId'], axis=1, inplace=True)

# Create
def get_titles(series):
    return series.str.extract(' ([a-zA-Z]+)\.', expand=False)

for dataset in data_all:
    dataset['FamilySize'] = dataset['Parch'] + dataset['SibSp'] + 1
    dataset['IsAlone'] = (dataset['FamilySize'] == 1).astype(int)
    
    dataset['Title'] = get_titles(dataset['Name'])
    title_counts = dataset['Title'].value_counts()
    dataset['Title'] = dataset['Title'].map(lambda t: t if title_counts[t] >= 10 else 'Rare')
    dataset.drop('Name', axis=1, inplace=True)
    
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)
    dataset['AgeBin'] = pd.cut(dataset['Age'], 5)

# Convert
categorical_cols = ['Sex', 'Embarked', 'Title', 'AgeBin', 'FareBin']
numerical_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']

le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)
for dataset in data_all:
    for col in categorical_cols:
        dataset[col + '_Code'] = le.fit_transform(dataset[col])

In [ ]:
def correlation_heatmap(df):
    plt.figure(figsize=(20, 10))
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(df.corr(), cmap=colormap, annot=True)
    plt.title('Pearson Correlation of Features', size=15)

In [ ]:
train = train.dropna(axis=0, subset=['Survived'])
targ_col = 'Survived'
# print('int & float cols:', train.columns[(train.dtypes == int) | (train.dtypes == float)].values)

feature_cols = ['Pclass', 'FamilySize', 'IsAlone', 'Sex_Code', 'Embarked_Code',
        'Title_Code', 'AgeBin_Code', 'FareBin_Code']
# print('chosen cols:', feature_cols)
# train[feature_cols].describe()

In [ ]:
classifiers = [
    KNeighborsClassifier(**{'n_jobs': -1, 'n_neighbors': 7}),
    SVC(**{'C': 0.390625, 'kernel': 'poly', 'probability': True, 'random_state': 0}),
    LinearSVC(**{'C': 25.0, 'dual': False, 'loss': 'squared_hinge', 'max_iter': 25000, 'penalty': 'l1', 'random_state': 0}),
    DecisionTreeClassifier(**{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 8, 'random_state': 0}),
    RandomForestClassifier(**{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 8, 'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}),
    AdaBoostClassifier(**{'algorithm': 'SAMME', 'learning_rate': 0.3, 'n_estimators': 300, 'random_state': 0}),jobs': -1, 'random_state': 0}),
    RidgeClassifier(**{'alpha': 0.5, 'normalize': True, 'random_state': 0}),
    GaussianProcessClassifier(**{'max_iter_predict': 10, 'n_jobs': -1, 'random_state': 0}),
    XGBClassifier(**{'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 10, 'n_jobs': -1, 'random_state': 0, 'reg_alpha': 0.16, 'reg_lambda': 2.56}),
    LGBMClassifier(**{'boosting_type': 'goss', 'learning_rate': 0.1, 'n_estimators': 1000, 'n_jobs': -1, 'random_state': 0, 'reg_alpha': 0.16, 'reg_lambda': 0}),
]

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
clf_scores = pd.DataFrame(columns=['Classifier', 'Test Score', 'Test Score 3*STD'])
clf_preds = pd.DataFrame(train[targ_col])
for i in range(len(classifiers)):
    clf = classifiers[i]
    clf_name = clf.__class__.__name__
    clf.fit(train[feature_cols], train[targ_col])
    
    cv_results = cross_val_score(clf, train[feature_cols], train[targ_col], cv=skf)s_') else feature_cols], train[targ_col], cv=skf)
    clf_scores.loc[i] = [clf_name, cv_results.mean(), cv_results.std() * 3]
    clf_preds[clf_name] = clf.predict(train[feature_cols])
clf_scores

In [ ]:
correlation_heatmap(clf_preds)

In [ ]:
def print_scores_info(model_name, scores):
    mean = scores.mean() * 100
    std_3 = scores.std() * 100 * 3
    print(model_name, 'score mean: ', mean)
    print(model_name, 'score 3 std range: ', mean - std_3, '—', mean + std_3)

In [ ]:
vote_classifiers = [
    ('knn', KNeighborsClassifier(**{'n_jobs': -1, 'n_neighbors': 7})),
    ('bag', BaggingClassifier(**{'max_samples': 0.1, 'n_estimators': 50, 'n_jobs': -1, 'random_state': 0})),
    ('gbc', GradientBoostingClassifier(**{'learning_rate': 0.3, 'max_depth': 2, 'min_samples_split': 2, 'n_estimators': 10, 'random_state': 0})),
    ('bnb', BernoulliNB(**{'alpha': 0.1})),
    ('gnb', GaussianNB()),
    ('xgb', XGBClassifier(**{'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 10, 'n_jobs': -1, 'random_state': 0, 'reg_alpha': 0.16, 'reg_lambda': 2.56})),
]

vote_hard_clf = VotingClassifier(vote_classifiers, voting='hard')

print_scores_info('Vote Hard', cross_val_score(vote_hard_clf, train[feature_cols], train[targ_col], cv=skf))

vote_hard_clf.fit(train[feature_cols], train[targ_col])
preds_test = vote_hard_clf.predict(data_test[feature_cols])

output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': preds_test})
output.to_csv('submission.csv', index=False)